### Exercise 4.2
#### Python challenge:

- Try implementing the GD version in NN code from scratch and see the effect of momentum and compare with SGD.

- Extend the neural network from scratch in python code to multi-step time series prediction. Use stocks from the NASDAQ stock market. Use Keras or scikit-learn to do the same and compare your results using Adam and SGD. 

In [2]:
import numpy as np 
import random
import matplotlib.pyplot as plt
from numpy import *
from sklearn import datasets 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import metrics 
# keras 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import roc_curve, auc
 

def read_data(run_num):
    # Source - raw and processed data:
    # https://github.com/sydney-machine-learning/Bayesianneuralnet_stockmarket/tree/master/code/dataset
    # five inputs (window size of 5) for 5 steps ahead (MMM dataset) https://finance.yahoo.com/quote/MMM/
    # code to process raw data:
    # https://github.com/sydney-machine-learning/Bayesianneuralnet_stockmarket/blob/master/code/data.py
    data = genfromtxt("datasets/MMM8_train.txt", delimiter=" ")
    data_X = data[:, 0:5]     # all features 0, 1, 2, 3, 4, 5, 6, 7 

    # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html
    # sklearn.preprocessing.Normalizer
    # transformer = Normalizer().fit(data_X)  # fit does nothing.
    # data_X = transformer.transform(data_X)
    data_y = data[:, 5:10]     # target - so that last col is selected from data

    X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.40, random_state=run_num)

    return X_train, X_test, y_train, y_test

    
def nn(X_train, X_test, y_train, y_test, type_model, hidden):
    timesteps = 5   # window size
    steps_ahead = 5 

    if type_model == 0:    # keras Adam
        nn = keras.Sequential()
        nn.add(layers.Dense(hidden, input_dim=timesteps, activation='relu'))
        nn.add(layers.Dense(steps_ahead, activation='sigmoid'))
        nn.compile(loss=keras.losses.binary_crossentropy,optimizer='adam', 
                   metrics=[keras.metrics.RootMeanSquaredError(), keras.metrics.mae,keras.metrics.mape])
         
    else:
        print('no model')    
  
    history = nn.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0)   # callbacks=[fa_test_his])
    y_pred_test = nn.predict(X_test)
    n = len(y_test)
    MAE = sum(np.abs(y_test - y_pred_test)) / n 
    RMSE_horizon1 = np.sqrt(sum(np.square(y_test[:,0] - y_pred_test[:,0])) / n)   # prediction horizon 1

    RMSE = np.sqrt(sum(np.square(y_test - y_pred_test)) / n) 
    MAPE = sum(np.abs((y_test - y_pred_test) / (y_test + 1e-6))) / n * 100  
   
    return RMSE, RMSE_horizon1, y_test, y_pred_test


def main(): 

    max_expruns = 2

    Adam_all = np.zeros(max_expruns) 
      
    max_hidden = 10
  
    for hidden in range(6, max_hidden, 2):
 
        for run_num in range(0, max_expruns): 
    
            X_train, X_test, y_train, y_test = read_data(run_num)   

            acc_adam, RMSE_horizon1, y_test, y_pred_test = nn(X_train, X_test, y_train, y_test, 0, hidden)  # Adam 
             
            Adam_all[run_num] = acc_adam
   
        print('RMSE all runs: ', Adam_all, hidden, '\n')
        print('Mean: ', np.mean(Adam_all), hidden, '\n')
        print('Std: ', np.std(Adam_all), hidden, '\n') 
 
    # next try a paragraph to describe your results and discuss which models are better to use.
    # repeat for another dataset
    # you can save results to a file as well
    
    
if __name__ == '__main__':
     main() 

RMSE all runs:  [0.04575978 0.07919058] 6 

Mean:  0.06247518425167685 6 

Std:  0.01671540058636813 6
RMSE all runs:  [0.04433717 0.08150803] 8 

Mean:  0.0629225999159567 8 

Std:  0.018585434295760588 8
